In [39]:
def morzesabc(txt):
    encrypted = {'A': '.-', 'B': '-...', 'C': '-.-.', 
                    'D': '-..', 'E': '.', 'F': '..-.', 
                    'G': '--.', 'H': '....','I': '..', 
                    'J': '.---', 'K': '-.-','L': '.-..',
                    'M': '--', 'N': '-.','O': '---', 
                    'P': '.--.','Q': '--.-','R': '.-.', 
                    'S': '...', 'T': '-','U': '..-', 
                    'V': '...-', 'W': '.--','X': '-..-',
                    'Y': '-.--', 'Z': '--..',
                    '1': '.----','2': '..---', '3': '...--',
                    '4': '....-', '5': '.....', '6': '-....',
                    '7': '--...', '8': '---..', '9': '----.',
                    '0': '-----', ', ': '--..--', '.': '.-.-.-',
                    '?': '..--..', '/': '-..-.', '-': '-....-',
                    '(': '-.--.', ')': '-.--.-'}
    
    decrypted = {v: k for k, v in encrypted.items()}

    if '-' in txt:
       return ''.join(decrypted[i] for i in txt.split())
    return ' '.join(encrypted[i] for i in txt.upper())

print(morzesabc("SOS"))
print(morzesabc('.--. -.-- - .... --- -.'))



... --- ...
PYTHON


In [1]:
import tkinter
import pytube

root = tkinter.Tk()
root.geometry("350x325")
root.title("YouTube Downloader")
root.resizable(0,0)
root.wm_minsize(0,0)


myVar = tkinter.StringVar()
myVar.set("YouTube Link:")
tkinter.Label(root, textvariable=myVar, width=50).pack(pady=5)

link = tkinter.StringVar()

def download():
   
        myVar.set("")
        root.update()
        pytube.YouTube(link.get()).streams.first().download(skip_existing=True, output_path="./")
        myVar.set("Great success".format(title_wrapped))
   

tkinter.Entry(root, textvariable=link, width=40).pack(pady=10)
tkinter.Button(root, text=f"Download!", command=download, fg="green", bg="white").pack()


root.mainloop()